In [1]:
import pandas as pd
import numpy as np
from catboost import Pool, CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit 
import optuna 
import json
from sklearn.metrics import mean_absolute_error

In [2]:
#Load inn datasets
X_test  = pd.read_parquet('../data/prepared_datasets/avg/no_duplicates/X_test.parquet')
X_train = pd.read_parquet('../data/prepared_datasets/avg/no_duplicates/X_train.parquet')
y_train = pd.read_parquet('../data/prepared_datasets/avg/no_duplicates/Y_train.parquet')
y_train_a = pd.read_parquet('../data/prepared_datasets/avg/no_duplicates/Y_train_a.parquet')
y_train_b = pd.read_parquet('../data/prepared_datasets/avg/no_duplicates/Y_train_b.parquet')
y_train_c = pd.read_parquet('../data/prepared_datasets/avg/no_duplicates/Y_train_c.parquet')

In [3]:
def test_train_split(df):
    dates_2 = (df.index >= '2023-04-01') & (df.index <= '2023-04-15')
    dates_1 = (df.index >= '2021-05-01') & (df.index <= '2021-08-01')

    test_set = df[dates_1 | dates_2]

    training_set = df[~(dates_1 | dates_2)]

    X_train = training_set.drop("pv_measurement", axis=1)
    y_train = training_set['pv_measurement']

    X_test = test_set.drop("pv_measurement", axis=1)
    y_test = test_set['pv_measurement'] 

    
    
    return X_train, X_test, y_train, y_test

X_train_new_a, X_test_new_a, y_train_new_a, y_test_a = test_train_split(pd.concat([X_train[X_train["location"] == "A"].drop("location", axis=1), y_train_a], axis=1))
X_train_new_b, X_test_new_b, y_train_new_b, y_test_b = test_train_split(pd.concat([X_train[X_train["location"] == "B"].drop("location", axis=1), y_train_b], axis=1))
X_train_new_c, X_test_new_c, y_train_new_c, y_test_c = test_train_split(pd.concat([X_train[X_train["location"] == "C"].drop("location", axis=1), y_train_c], axis=1))

In [4]:
#Create a pool of data
train_pool_a = Pool(X_train_new_a, y_train_new_a)
train_pool_b = Pool(X_train_new_b, y_train_new_b)
train_pool_c = Pool(X_train_new_c, y_train_new_c)


test_pool_a = Pool(X_test_new_a) 
test_pool_b = Pool(X_test_new_b) 
test_pool_c = Pool(X_test_new_c) 

In [ ]:
#For location A
def objective(trial, X_train, y_train):
    params = {
        "iterations": trial.suggest_int("iterations", 300, 3500),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 13),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 2, 10),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.3, 1.0),
        "random_strength": trial.suggest_float("random_strength", 0.3, 1.0),
        "border_count": trial.suggest_int("border_count", 1, 1000),
        "rsm": trial.suggest_float("rsm", 0.05, 1),
        "loss_function" :"LogCosh"
    }

    catboost_model_a = CatBoostRegressor(**params, verbose=300)
    catboost_model_a.fit(train_pool_a)
    pred_a = pd.DataFrame(catboost_model_a.predict(test_pool_a))
    MAE_a = mean_absolute_error(y_test_a, pred_a)
    return MAE_a

study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=80)

[I 2023-11-11 09:29:12,137] A new study created in memory with name: no-name-4c3530c0-8e9d-4e86-8e65-335cecc85848


0:	learn: 572.0497150	total: 75.2ms	remaining: 2m 5s
300:	learn: 371.9154225	total: 6.74s	remaining: 30.8s
600:	learn: 270.4254772	total: 13.5s	remaining: 24.1s
900:	learn: 224.8173899	total: 20.4s	remaining: 17.5s
1200:	learn: 203.5769002	total: 27.4s	remaining: 10.8s
1500:	learn: 193.3007018	total: 34.5s	remaining: 3.97s


[I 2023-11-11 09:29:50,943] Trial 0 finished with value: 352.9731565896283 and parameters: {'iterations': 1674, 'learning_rate': 0.0020336134864032264, 'depth': 5, 'min_data_in_leaf': 18, 'l2_leaf_reg': 5, 'bagging_temperature': 0.7118574372705158, 'random_strength': 0.7071908896762809, 'border_count': 995, 'rsm': 0.9545408657511946}. Best is trial 0 with value: 352.9731565896283.


1673:	learn: 189.9468448	total: 38.5s	remaining: 0us
0:	learn: 570.5091056	total: 16.9ms	remaining: 47.1s
300:	learn: 232.9667114	total: 6.23s	remaining: 51.5s
600:	learn: 185.0585845	total: 14.6s	remaining: 53.2s
900:	learn: 175.6354239	total: 26.2s	remaining: 54.8s
1200:	learn: 171.6515047	total: 37.8s	remaining: 49.9s
1500:	learn: 168.8334451	total: 49.9s	remaining: 42.8s
1800:	learn: 166.4024903	total: 1m 1s	remaining: 34s
2100:	learn: 164.0092386	total: 1m 13s	remaining: 24.2s
2400:	learn: 161.6883766	total: 1m 25s	remaining: 13.9s
2700:	learn: 159.6381367	total: 1m 37s	remaining: 3.18s


[I 2023-11-11 09:31:32,646] Trial 1 finished with value: 306.86800279689265 and parameters: {'iterations': 2789, 'learning_rate': 0.005215719786221631, 'depth': 7, 'min_data_in_leaf': 23, 'l2_leaf_reg': 9, 'bagging_temperature': 0.59927201488059, 'random_strength': 0.45078026156259454, 'border_count': 177, 'rsm': 0.28375386640818895}. Best is trial 1 with value: 306.86800279689265.


...

2524:	learn: 153.9722980	total: 58.3s	remaining: 0us
0:	learn: 569.0599542	total: 47.2ms	remaining: 2m 27s
300:	learn: 189.1156263	total: 14.4s	remaining: 2m 14s
600:	learn: 163.6785437	total: 29.1s	remaining: 2m 1s
900:	learn: 156.8344887	total: 43.7s	remaining: 1m 47s
1200:	learn: 151.3138823	total: 58.6s	remaining: 1m 33s
1500:	learn: 146.9484733	total: 1m 13s	remaining: 1m 19s
1800:	learn: 141.4168611	total: 1m 28s	remaining: 1m 4s
2100:	learn: 134.9961274	total: 1m 43s	remaining: 49.9s
2400:	learn: 129.5287600	total: 1m 58s	remaining: 35.3s
2700:	learn: 125.9350808	total: 2m 13s	remaining: 20.6s
3000:	learn: 121.1178977	total: 2m 28s	remaining: 5.79s


[I 2023-11-11 11:42:44,598] Trial 77 finished with value: 307.1805007896943 and parameters: {'iterations': 3118, 'learning_rate': 0.008018935622246951, 'depth': 9, 'min_data_in_leaf': 3, 'l2_leaf_reg': 3, 'bagging_temperature': 0.40417874797170517, 'random_strength': 0.4344471236090092, 'border_count': 842, 'rsm': 0.8081469763409589}. Best is trial 32 with value: 303.9605587407062.


3117:	learn: 119.4035631	total: 2m 34s	remaining: 0us
0:	learn: 569.4836837	total: 35ms	remaining: 1m 42s
300:	learn: 185.0733268	total: 11.2s	remaining: 1m 38s
600:	learn: 167.2597114	total: 22.8s	remaining: 1m 28s
900:	learn: 161.8584449	total: 34.2s	remaining: 1m 17s
1200:	learn: 156.8674063	total: 45.6s	remaining: 1m 5s
1500:	learn: 150.8820608	total: 57.2s	remaining: 54.7s
1800:	learn: 144.9102947	total: 1m 8s	remaining: 43.4s
2100:	learn: 140.9946888	total: 1m 20s	remaining: 32s
2400:	learn: 137.7877759	total: 1m 32s	remaining: 20.6s
2700:	learn: 133.8688914	total: 1m 43s	remaining: 9.09s


[I 2023-11-11 11:44:38,395] Trial 78 finished with value: 304.6653821545268 and parameters: {'iterations': 2937, 'learning_rate': 0.009142133921305558, 'depth': 8, 'min_data_in_leaf': 8, 'l2_leaf_reg': 2, 'bagging_temperature': 0.3536170804061364, 'random_strength': 0.48691990291411724, 'border_count': 919, 'rsm': 0.8887455218709256}. Best is trial 32 with value: 303.9605587407062.


2936:	learn: 130.9753483	total: 1m 53s	remaining: 0us
0:	learn: 568.4889967	total: 16.3ms	remaining: 38.1s
300:	learn: 185.2567902	total: 5.49s	remaining: 37.3s
600:	learn: 174.0849992	total: 11.2s	remaining: 32.3s
900:	learn: 169.3675208	total: 16.9s	remaining: 27.1s
1200:	learn: 164.8166608	total: 22.8s	remaining: 21.7s
1500:	learn: 161.0385552	total: 28.7s	remaining: 16.1s
1800:	learn: 157.8099406	total: 34.6s	remaining: 10.4s
2100:	learn: 154.9781069	total: 40.7s	remaining: 4.71s


In [10]:
#to output the best paramaters
print(study.best_params)

#to output the best score returned from the trials
print(study.best_value)


with open("optuna-best-parameters_a.txt", "w") as file:
    file.write("Best paramaters: \n")
    file.write(json.dumps(study.best_params))  # Write the first string followed by a newline character
    file.write("\n")
    file.write("best score MAE: \n")
    file.write(json.dumps(study.best_value))  # Write the second string followed by a newline character

{'iterations': 1852, 'learning_rate': 0.0018854412552105353, 'depth': 9, 'colsample_bylevel': 0.7366562974962199, 'min_data_in_leaf': 4, 'l2_leaf_reg': 3, 'has-time': True, 'bagging_temperature': 0.9525931510378696, 'random_strength': 0.30613129187014626, 'border_count': 993, 'rsm': 0.21119141087462187, 'nan_mode': 'min'}
351.34628571238125
